# Earthquake Data Analysis

### Description

The catalog includes the magnitude, time of occurrence (s), and 3D coordinates (m) of earthquakes in about 20 years of recording in South California. Coordinates were converted from latitude, longitude, and depth of events in a seismic catalog. Magnitudes should be within the range $[0,8]$.

* **Waiting time (t)**: time interval between an event and the next one in the sequence.
* **Distance (r)**: Eucledian 3D distance between events. (each 3D set of coordinates refers to the hypocenter, i.e. the point triggering the slip in a fault that forms the earthquake)


### Assignments

1. Deduce what is the variable in each column of the catalog.
2. Visualize the process in space and/or time with suitable time series and/or 3D visualizations of the hypocenters. For instance, plot a space variable (a single coordinate or a nice linear combination of coordinates) as a function of time.
3. Compute the distribution $P_m(t)$ of waiting times for events of magnitude m or above (i.e. do not consider events below $m$). In shaping the bin sizes, take into account that this distribution is expected to have a power-law decay with time (e.g $\sim 1/t$), and that a power-law is well visualized in log-log scale. Do this analysis for many values of $m$, say $m=2,3,4,5$.
4. Compute the distribution $P_m(r)$ of the distance between an event and the next one, considering earthquakes of magnitude m or above. Also here make a clever choice for the bin sizes and try several values of $m$.
5. Compute the distribution $P_{m,R}(t)$ of waiting times for events of magnitude $m$ or above, which are separated by at most a distance $r<R$, for different values of m and $R$. (In this statistics, if the following event is farther than $R$, skip the $t$ and go to the next pair)
6. Eventually note if, from the analysis of the previous points, there emerges a scaling picture. Is there a suitable rescaling that collapses distributions for various $m$ (and eventually $R$ if point 5 is considered) on a single curve?

### Datasets

* column 1: index of the event
* column 2: index of the previous event that triggered it (defined with a given algorithm), -1 if no ancestor is found
* column 3: time (seconds) from 0:00 of Jan.1st, 1982
* column 4: magnitude
* columns 5, 6, and 7: 3D coordinates (meters) of the earthquake hypocenter, i.e. of the point from where it started. These Euclidean coordinates are derived from latitude, longitude and depth.

Joining each event to that with the index of the second column (if not -1), there emerges a set of causal trees.


### Contact
* Marco Baiesi <marco.baiesi@unipd.it>

In [84]:
%matplotlib inline

#General Libraries

import time



import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

#PROJECTIONS LIBRARY
import pyproj

# Map library
import folium
from folium import plugins

#Colormap library
from branca.colormap import linear

# Image Processing
from PIL import Image
import io
from selenium import webdriver
import imageio
import os

from pathlib import Path



In [114]:
header = ['ID', 'ID_previous_event', 'time', 'Magnitude', 'X', 'Y', 'Z']
df = pd.read_csv('./SouthCalifornia-1982-2011_Physics-of-Data.dat', sep=' ', names=header, index_col=0)
df.head()

,ID_previous_event,time,Magnitude,X,Y,Z
ID,,,,,,
0,-1,0.00000,2.71,-2571956,-4627162,3520602
1,0,36501.39072,2.12,-2363740,-4787011,3461373
2,0,37488.27744,2.33,-2363746,-4786942,3461232
3,0,47982.51648,2.57,-2475085,-4664024,3548479
4,0,60268.57056,2.98,-2238642,-4839098,3469546


In [115]:
time1 = datetime.datetime(1982, 1, 1, 0, 0, 0)
time2 = datetime.datetime(1970, 1, 1, 0, 0, 0)

delta = time1 - time2

total_seconds = delta.total_seconds()

#df["timestamp"] = pd.to_datetime(df["time"], unit="s").dt.strftime("%Y-%m-%d %H:%M:%S")
df["timestamp"] = pd.to_datetime(df["time"] + total_seconds, unit="s").dt.strftime("%Y-%m-%dT%H:%M:%S")

In [116]:
df.head()

,ID_previous_event,time,Magnitude,X,Y,Z,timestamp
ID,,,,,,,
0,-1,0.00000,2.71,-2571956,-4627162,3520602,1982-01-01T00:00:00
1,0,36501.39072,2.12,-2363740,-4787011,3461373,1982-01-01T10:08:21
2,0,37488.27744,2.33,-2363746,-4786942,3461232,1982-01-01T10:24:48
3,0,47982.51648,2.57,-2475085,-4664024,3548479,1982-01-01T13:19:42
4,0,60268.57056,2.98,-2238642,-4839098,3469546,1982-01-01T16:44:28


In [117]:
# Definition of the two reference frames 
ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84') #euclidean
lla  = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84') #latlon

# Coordinates conversion
points = np.array([df.X, df.Y, df.Z])
gcoor  = pd.DataFrame(zip(*(pyproj.transform(ecef, lla, points[0], points[1], points[2], radians=False))))

gcoor.columns = ['longitude', 'latitude', 'depth']
df = pd.concat([df, gcoor], axis=1)

/tmp/ipykernel_23473/1083525435.py:7: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  gcoor  = pd.DataFrame(zip(*(pyproj.transform(ecef, lla, points[0], points[1], points[2], radians=False))))


$$X = (N(\phi)+h) \space cos\phi \space cos\lambda $$
$$Y = (N(\phi)+h) \space cos\phi \space sin\lambda $$
$$Z = \left( \frac {b^2} {a^2} N(\phi) + h \right) sin\phi $$

where:

$$ N(\phi) = \frac {a^2} {\sqrt{a^2 cos\phi^2 + b^2 sin\phi^2}} = 
\frac {a} {\sqrt{1 - e^2 sin\phi ^2}}
$$




Source: <a href="https://en.wikipedia.org/wiki/Geographic_coordinate_conversion#:~:text=In%20geodesy%2C%20geographic%20coordinate%20conversion,translation%20among%20different%20geodetic%20datums.">Geographic Coordinate Conversion</a>

In [118]:
df.tail()

,ID_previous_event,time,Magnitude,X,Y,Z,timestamp,longitude,latitude,depth
110266,-1,9.304996e+08,2.60,-2668492,-4335735,3810743,2011-06-27T16:06:08,-121.610832,37.000291,-11092.891170
110267,-1,9.305115e+08,2.02,-2297480,-4823870,3445285,2011-06-27T19:25:04,-115.467209,32.990515,-14309.251619
110268,-1,9.305318e+08,2.00,-2404797,-4441247,3868121,2011-06-28T01:03:15,-118.434166,37.634193,-8595.471379
110269,-1,9.305363e+08,2.17,-2388375,-4691191,3550903,2011-06-28T02:17:48,-116.981514,34.180503,-21582.249477
110270,-1,9.305667e+08,3.27,-2579453,-4409462,3774036,2011-06-28T10:45:35,-120.326838,36.640673,-19161.872200


In [123]:

linear.YlOrRd_09.colors.reverse()

colormap = linear.YlOrRd_09.scale(-32000, 0)
colormap = colormap.to_step(index=[-32000, -25000, -20000, -15000, -10000, -5000, 0])
colormap.caption = "Depth of the hypocenter (m)"



df["year"] = pd.DatetimeIndex(df['timestamp']).year


for i in df["year"].unique():
    m = folium.Map(location=[35.496933, -119.417933], zoom_start=6)
    folium.TileLayer("Stamen Terrain").add_to(m)
    colormap.add_to(m)
    
    df_i = df[df["year"] == i]
    for _,row in df_i.iterrows():
        color = colormap(row["depth"] )
        folium.CircleMarker(location = [row["latitude"], row["longitude"]], radius=row["Magnitude"], 
                        color=color, fill=True, fill_color=color,
                        fill_opacity=1, popup= row["depth"]).add_to(m)
    title_html = '''
                 <h3 align="center" style="font-size:22px"><b>{}</b></h3>
                 '''.format('Year: ' + str(i))   
    m.get_root().html.add_child(folium.Element(title_html))

    m.save(f"map_{i}.html")
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f"file:///home/joan/PoD/LCP_A/LCP_A-Project/LCP_projects_Y5-Group13/map_{i}.html")
    time.sleep(5)
    img_data = driver.get_screenshot_as_png()
    driver.quit()
    img = Image.open(io.BytesIO(img_data))
    img = img.convert("RGB")
    img.save(f'map_{i}.jpeg', quality=95)

    image = Image.open(f'map_{i}.jpeg')
    box = (50, 0, 780, 580)
    cropped_image = image.crop(box)
    cropped_image.save(f'map_{i}.jpeg')

image_path = Path()
images = list(image_path.glob("*.jpeg"))
images.sort()
html_files = list(image_path.glob("*.html"))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
    os.remove(file_name)
for file in html_files:
    os.remove(file)
imageio.mimwrite("GifMap.gif", image_list, fps=2)